## Name Convention in LIPS to have a more dynamic variable naming
This has been done only for power grid use case for the moment

In [ ]:
import pathlib
from lips import get_root_path
from lips.config.configmanager import ConfigManager

In [ ]:
LIPS_PATH = get_root_path(pathlib_format=True).parent#pathlib.Path().resolve().parent
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_case14_sandbox"
LOG_PATH = LIPS_PATH / "lips_logs.log"

### Read the config file including the names convention for attributes

In [ ]:
NAME_CONVENTION_CONFIG = LIPS_PATH / "configurations" / "powergrid" / "names_convention.ini"

In [ ]:
from lips.config.configmanager import ConfigManager

config = ConfigManager(NAME_CONVENTION_CONFIG, "DEFAULT")
print(config)

### Use the NameConvention class to load the attributes

In [ ]:
from lips.dataset.utils.powergrid_utils import NamesConvention

my_convention = NamesConvention()
print(my_convention)

You may change the names convention directly

In [ ]:
my_convention.input_variables_title = "attr_x"

You may also load the names convention from a config file

In [ ]:
my_convention.set_from_config(config)

In [ ]:
my_convention.todict()

### Use an example to check if everything works properly

In [ ]:
from lips.benchmark.powergridBenchmark import PowerGridBenchmark

benchmark = PowerGridBenchmark(benchmark_path=None,
                               benchmark_name="Benchmark3",
                               config_path=BENCH_CONFIG_PATH,
                               load_data_set=False,
                               names_convention=my_convention,
                               log_path=LOG_PATH
                               )

In [ ]:
benchmark.generate(nb_sample_train=int(1e2),
                   nb_sample_val=int(1e2),
                   nb_sample_test=int(1e2),
                   nb_sample_test_ood_topo=int(1e2),
                   do_store_physics=True,
                  )

In [ ]:
benchmark.train_dataset.data.keys()

Train a model

In [ ]:
from lips.augmented_simulators.torch_models.fully_connected import TorchFullyConnected
from lips.augmented_simulators.torch_simulator import TorchSimulator
from lips.dataset.scaler import StandardScaler

torch_sim = TorchSimulator(name="torch_fc",
                           model=TorchFullyConnected,
                           scaler=StandardScaler,
                           log_path=LOG_PATH,
                           device="cpu",
                           seed=42,
                           bench_config_path=BENCH_CONFIG_PATH,
                           bench_config_name="Benchmark1",
                           sim_config_path=SIM_CONFIG_PATH / "torch_fc.ini",
                           sim_config_name="DEFAULT" # use the default set of hyper parameters
                          )

In [ ]:
torch_sim.train(benchmark.train_dataset, benchmark.val_dataset, save_path=None, epochs=20, train_batch_size=128)

In [ ]:
torch_sim_metrics = benchmark.evaluate_simulator(augmented_simulator=torch_sim,
                                                 eval_batch_size=128,
                                                 dataset="all",
                                                 shuffle=False,
                                                 save_path=None,
                                                 save_predictions=False
                                                )

In [ ]:
torch_sim_metrics["test"]["Physics"]